In [ ]:
import json
from tqdm import tqdm
import numpy as np
import pandas as pd

#Create Dataset

In [ ]:
target_filepath = "" #TODO
drive_path = "" #TODO

Import geojson files

In [ ]:
def open_json(json_path):
  with open(json_path) as f:
    data = json.load(f)
  return data

iri_data = open_json(drive_path + "/02 Development/data/99 meta/IRI_201908.geojson")["features"]
img_data = open_json(drive_path + "/02 Development/data/99 meta/Aufnahmepunkte_GeoJPG_201908 mit Straßentyp.geojson")["features"]
rides_data = open_json(drive_path + "/02 Development/data/99 meta/Befahrung_201908.geojson")["features"]

Recorded rides with sensor vehicle

In [ ]:
rides = [i["properties"]["_VIDEO"] for i in rides_data]
rides

['20190819_12-50-08-839',
 '20190814_14-44-24-433',
 '20190819_10-32-50-611',
 '20190819_12-00-05-082',
 '20190816_10-30-47-864',
 '20190819_11-15-57-991',
 '20190816_11-08-09-342',
 '20190819_12-26-11-227',
 '20190816_10-09-20-137',
 '20190819_13-39-58-707',
 '20190819_13-48-34-589',
 '20190819_13-57-11-762',
 '20190819_14-39-18-092']

#Connect IRI data to image data
In the recorded data each IRI value is assigned to a road segment.
In this function, the three closest segments to an image are calculated. The IRI value of the upcoming segment is offset by applying weights based on the distance to the segment

In [ ]:
import numpy as np

# Get indices of the images that are closest to the iri segment of the given ride
def get_k_closest_images(segm_coordinates, ride_video, img_data, k=1):  
  p1 = np.array(segm_coordinates[0])
  p2 = np.array(segm_coordinates[1])
  distances = []
  ride_imgs = []

  for img in img_data:
    if img["properties"]["DATE"]+"_"+img["properties"]["TIME"] == ride_video:
      ride_imgs.append(img)

      p3 = np.array(img["geometry"]["coordinates"][0:2])    
      distance = abs(np.linalg.norm(p1-p3) + np.linalg.norm(p2-p3))
      distances.append(distance)
  distances = np.array(distances)
  idx = np.argpartition(distances, k)
  print(distances[idx[:k]])
  print(idx[:k])

  print([ride_imgs[i] for i in idx[:k]])
  result_image_names = [ride_imgs[i]["properties"]["NAME"] for i in idx[:k]]
  return result_image_names

def get_iri_for_closest_segment(img, ride_video, iri_data, front_weighted_calculation = False):
  p0 = np.array(img["geometry"]["coordinates"][0:2])
  closest_segm = {"dist":100,
             "segm":0}
  sec_closest_segm = {"dist":100,
                 "segm":0}
  thi_closest_segm = {"dist":100,
                 "segm":0}

  # calculate IRI segment that is closest to the image position
  for d in iri_data:
    if d["properties"]["VIDEO"] == ride_video:
      if d["properties"]["IRIC_CLASS"] != "NV":
        p1 = np.array(d["geometry"]["coordinates"][0])
        p2 = np.array(d["geometry"]["coordinates"][1])
        m = (p2 - p1) / 2 + p1
        distance = abs(np.linalg.norm(p0-m))
        if distance < closest_segm["dist"]:
          thi_closest_segm = sec_closest_segm
          sec_closest_segm = closest_segm
          closest_segm = {"dist" : distance,
                    "segm" : d}
        elif distance < sec_closest_segm["dist"]:
          thi_closest_segm = sec_closest_segm
          sec_closest_segm = {"dist" : distance,
                        "segm" : d}
        elif distance < thi_closest_segm["dist"]:
          thi_closest_segm = {"dist" : distance,
                        "segm" : d}
  unsorted_segmts = [closest_segm,sec_closest_segm,thi_closest_segm]
  
  # unused approach of calculating the IRI value of the IRI segment ahead of the image position
  if front_weighted_calculation:
    sorted_segmts = sorted(unsorted_segmts, key=lambda k: k["segm"]['properties']["Abnr"])

    iri_1 = int(sorted_segmts[0]["segm"]["properties"]["IRIC_CLASS"])
    iri_2 = int(sorted_segmts[1]["segm"]["properties"]["IRIC_CLASS"])
    iri_3 = int(sorted_segmts[2]["segm"]["properties"]["IRIC_CLASS"])
    d1 = float(sorted_segmts[0]["dist"])
    d2 = float(sorted_segmts[1]["dist"])
    d3 = float(sorted_segmts[2]["dist"])

    # Calculation to get IRI value of segment in front of image
    if d1 <= d3:
      x = (d1-d2)/(d2+d1)
    elif d1 > d3:
      x = (d3-d2)/(d3+d2) + 0.5

    weight_2 = min(max(1 - x,0),1)
    weight_3 = min(max(x,0),1)

    iri_class_result = (iri_2 * weight_2 + iri_3 * weight_3) / (weight_2 + weight_3)
    distance_result = d2
    return iri_class_result, distance_result
  else:
    # Use IRI value of closest segment
    iri_class_result = int(unsorted_segmts[0]["segm"]["properties"]["IRIC_CLASS"])
    iri_cont_result = float(unsorted_segmts[0]["segm"]["properties"]["IRIC_m_km"])
    distance_result = float(unsorted_segmts[0]["dist"])
  
  return iri_class_result, iri_cont_result, distance_result

#Run function and save csv fie


In [ ]:
datapoints = []

ROUND_IRI_VALUES = True

# Iterating rides to ensure distinct IRI assignment
for ride_video in tqdm(rides):
  ride_imgs = [i for i in img_data if i["properties"]["DATE"]+"_"+i["properties"]["TIME"] == ride_video]
  for img in ride_imgs:
    iri_class,iri_cont,dist = get_iri_for_closest_segment(img, ride_video, iri_data)
    if dist < 0.00014:
      img_name = img["properties"]["NAME"]
      if ROUND_IRI_VALUES:
        iri_class = round(iri_class)
      datapoints.append([img_name,iri_class,iri_cont])
df_data = pd.DataFrame(datapoints)

df_data.to_csv(target_filepath,index = False,header = False)